# CSC299: Summer 2021: Report 1 - Implementation of LCU with 1 Ancilla

## Arkaprava Choudhury

In [1]:
import tequila as tq
import copy
from typing import Iterable
import numpy as np

## Procedure

Consider the Hamiltonian $H$ which acts over $n$ qubits. Let $N = 2^n$. WIthout loss of generality, we can assume that $H$ is Hermitian. Indeed, suppose that $H$ was not Hermitian. Then, we would define $\tilde{H}$ such that:
\begin{align*}
    \tilde{H} &=
    \begin{bmatrix}
        0 & H^\dagger \\
        H & 0
    \end{bmatrix}
\end{align*}
Thus, $\tilde{H}$ acts over $2n$ qubits, where the first $n$ qubits are ancillae. Furthermore, as is evident by the construction above, $\tilde{H}$ is Hermitian, even though $H$ is not.

Suppose that we have the decomposition $H = \sum_{j=0}^{m-1} \alpha_j U_j$ over $n$ qubits, where each of the $U_j$'s are unitary. We know that any arbitrary Hamiltonian can be expressed in such a form due to the fact that any element of the standard orthonormal basis can be expressed as a linear combination of such unitary operators (see Appendix for detailed proof).

Let $s = \sum_{k=0}^{m-1} \alpha_k$. Moreover, without loss of generality, we can assume that $\alpha_j > 0$ for all $j$. Indeed, suppose that there exists some $\alpha_j < 0$: we can then replace the term $\alpha_j U_j$ with the term $\alpha^\prime_j U^\prime_j$, where $(\alpha^\prime_j, U^\prime_j) = (-\alpha_j, - U_j)$.

We define the $\text{Prepare}$ oracle as follows:
\begin{align*}
    \text{Prepare}\left| 0 \right\rangle &= \frac1{\sqrt{s}} \sum_{j=0}^{m-1} \sqrt{\alpha_j} \left| j \right\rangle
\end{align*}
We also define the $\text{Select}$ oracle as follows:
\begin{align*}
    \text{Select}\left| j \right\rangle \left| \psi \right\rangle &= \left| j \right\rangle U_j \left| \psi \right\rangle
\end{align*}

We thus define the operator $W$ as follows:
\begin{align*}
    W &= (\text{Prepare}^\dagger\otimes I_N) \text{Select} (\text{Prepare}\otimes I_N)
\end{align*}
Therefore, we see that:
\begin{align*}
    W \left| 0 \right\rangle \left| \psi \right\rangle &= \frac1{s} \left| 0 \right\rangle H\left| \psi \right\rangle + \sqrt{1-\frac1{s^2}} \left| \phi \right\rangle
\end{align*}
where $\left| \phi \right\rangle$ is some state whose ancillary component lies in a subspace orthogonal to $\left| 0 \right\rangle$. Hence, if $P_0 = \left| 0 \right\rangle \left\langle 0 \right|\otimes I_N$ is the projector onto the zero-subspace, then we have that:
\begin{align*}
    P_0W\left| 0\right\rangle\left| \psi\right\rangle &= \frac1{s} \left| 0 \right\rangle H\left| \psi \right\rangle
\end{align*}

The quantum circuit for this procedure looks as shown below:

TODO: ADD DIAGRAM

To obtain an exact implementation of $H$, we can choose to use amplitude amplification at the end of this procedure.

For more details about these equations, refer Appendix.

### Notes on procedure

Notice that when $H$ can be written as a linear combination of $m$ unitary operators with all positive coefficients, then we need a total of $k = \lceil \log_2 m\rceil ancillary qubits to implement the above procedure.

Also note that the success probability of emulating $H$ by the above procedure is $\frac1{s^2}$. This probability depends on the size of the ancilla and decreases exponentially in the number of qubits contained in the ancilla. We can apply the procedure of amplitude amplification (refer Appendix) to increase this probability up to 1. Ideally, if $s=2$, then this fits the procedure of oblivious amplitude amplification. However, since $H$ is not unitary itself, we need to slightly alter the approach of oblivious amplitude amplification that we use here.

Firstly, define the $m$-dimensional ancillary reflection $R = I_m - 2 P_0$. Next, define the amplitude amplification operator $A = - WRW^\dagger RW$.

Recall the following facts. Firstly, note that $P_0^2 = P_0$ since $P_0$ is a projection operator and that $P_0 \left| 0 \right\rangle \left| \psi \right\rangle = \left| 0 \right\rangle\left| \psi \right\rangle$. Also note that, by definition, $W$ is a unitary operator. Moreover, recall our assumption that $H$ is Hermitian. Finally, by construction, we have that:
\begin{align*}
    PWP &= \frac1{s} (\left| 0 \right\rangle\left\langle 0 \right|\otimes H)
\end{align*}

Therefore, we observe that:
\begin{align*}
    PA\left| 0 \right\rangle\left| \psi \right\rangle &= \left(\frac3{s}-\frac4{s^3}\right) \left| 0 \right\rangle H \left| \psi \right\rangle
\end{align*}

### One qubit example

Suppse that we are given a Hamiltonian $H$ as follows:
\begin{align*}
    H &=
    \begin{bmatrix}
        0 & 0 \\
        0 & 1
    \end{bmatrix}
\end{align*}
Then, clearly, $H$ is non-unitary, since $HH^\dagger\neq I_2$. However, we can write $H = \frac1{2} I - \frac1{2} Z$, thus expressing $H$ as the sum of two unitary operations.



## Implementation of one qubit ancilla 

In this section, we wish to create an implementation of the LCU procedure 

We first need to define a subroutine to return the controlled version of the unitary. We do so as follows:

In [2]:
def control_unitary(ancilla, unitary: tq.QCircuit) -> tq.QCircuit:
    """Return controlled version of unitary

    SHOULD NOT mutate unitary

    Preconditions:
        - ancilla and unitary have no common qubits
    """
    gates = unitary.gates
    cgates = []
    for gate in gates:
        cgate = copy.deepcopy(gate)
        if isinstance(ancilla, Iterable):
            control_lst = list(cgate.control) + list(ancilla)
        else:
            control_lst = list(cgate.control) + [ancilla]
        cgate._control = tuple(control_lst)
        cgate.finalize()
        cgates.append(cgate)

    return tq.QCircuit(gates=cgates)

Next, we implement the prepare oracle for the single-qubit ancilla case in the form of an $R_y$ rotation as follows.

In [8]:
def prepare_2unitary(ancillary, sum_of_unitaries: list[tuple[float, tq.QCircuit]]) -> tq.QCircuit:
    """
    Prepare operator, when the Hamiltonian can be expressed as the linear combination of two
    unitary operators.
    Requires only one ancillary qubit.

    Preconditions:
        - ...
    """
    alpha_0, alpha_1 = sum_of_unitaries[0][0], sum_of_unitaries[1][0]

    theta = -2 * np.arcsin(np.sqrt(alpha_1 / (alpha_0 + alpha_1)))

    return tq.gates.Ry(target=ancillary, angle=theta)

The select oracle is naturally translated into code as follows.

In [4]:
def select_2unitary(ancillary, unitary_0: tq.QCircuit, unitary_1: tq.QCircuit) -> tq.QCircuit:
    """
    Select operator, when the Hamiltonian can be expressed as the linear combination of two
    unitary operators.
    Requires only one ancillary qubit.

    Returns the select oracle.
    """
    impl_1 = control_unitary(ancilla=ancillary, unitary=unitary_1)
    
    x_gate = tq.gates.X(target=ancillary)
    control = control_unitary(ancilla=ancillary, unitary=unitary_0)

    impl_0 = x_gate + control + x_gate

    return impl_1 + impl_0

In [5]:
def algorithm_2unitary(unitaries: list[tuple[float, tq.QCircuit]]) -> tq.QCircuit:
    """The main part.

    TODO: write proper docstring
    """
    prepare = prepare_2unitary(0, unitaries)
    circ = prepare + select_2unitary(0, unitaries[0][1], unitaries[1][1]) + prepare.dagger()
    return circ

### Notes on implementation

Input, output as circuits. Advantage: independent of starting state, so if initial state can be efficiently prepared, then we can efficiently simulate Hamiltonian. Drawback: need to have the circuits for each unitary used prepared beforehand.

### Example

In [11]:
def example_function() -> tq.QCircuit:
    """Test example for LCU with two unitary matrices"""
    identity = tq.QCircuit()
    unitaries = [(0.5, identity), (0.5, tq.gates.Z(1))]
    return algorithm_2unitary(unitaries=unitaries)

In [12]:
result = example_function()
tq.draw(result, backend='qiskit')

┌──────────┐   ┌───┐┌───┐┌─────────┐
q_0: ┤ RY(-π/2) ├─■─┤ X ├┤ X ├┤ RY(π/2) ├
     └──────────┘ │ └───┘└───┘└─────────┘
q_1: ─────────────■──────────────────────
                                         
c: 2/════════════════════════════════════

# Next steps: 3 or 4 unitaries (2 ancillae), and then, m unitaries